In [2]:
import json
from datetime import datetime
import pandas as pd
from openai_query import consultar_chatgpt
import duckdb as ddbb
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings("ignore", category=SyntaxWarning)

def get_d_minus_1_timestamps_unix():
    # Data e hora atual
    hoje = datetime.now()
    
    # Data de D-1
    d1 = hoje - timedelta(days=1)
    
    # Limites do dia D-1 (00:00:00 e 23:59:59)
    d1_min = d1.replace(hour=0, minute=0, second=0, microsecond=0)
    d1_max = d1.replace(hour=23, minute=59, second=59, microsecond=999999)
    
    # Converte para timestamp UNIX (inteiros)
    ts_min = int(d1_min.timestamp())
    ts_max = int(d1_max.timestamp())
    
    # Data atual no formato yyyymmdd
    data_atual = hoje.strftime("%Y%m%d")
    
    return data_atual, ts_min, ts_max

# Exemplo de uso:
data_atual, ts_min, ts_max = get_d_minus_1_timestamps_unix()

In [3]:
con = ddbb.connect()
con.install_extension('json')
con.load_extension('json')
# con.create_function('consultar_chatgpt', consultar_chatgpt)
# con.execute("INSTALL excel;")
# con.execute("LOAD excel;")

In [44]:
#CAMADA BRONZE
path = r'landing\*.json'

# Usa DuckDB para salvar diretamente em parquet, excluindo o campo ID
con.execute(f'''
    COPY (
        SELECT 
            id, timestamp, "from", fromMe, source, body, "to", participant,
            hasMedia, media, ack, ackName, replyTo,
            -- Reconstrói o struct _data sem o campo Info.ID
            {{
                'Info': {{
                    'AddressingMode': _data.Info.AddressingMode,
                    'BroadcastListOwner': _data.Info.BroadcastListOwner,
                    'BroadcastRecipients': _data.Info.BroadcastRecipients,
                    'Category': _data.Info.Category,
                    'Chat': _data.Info.Chat,
                    'DeviceSentMeta': _data.Info.DeviceSentMeta,
                    'Edit': _data.Info.Edit,
                    'IsFromMe': _data.Info.IsFromMe,
                    'IsGroup': _data.Info.IsGroup,
                    'MediaType': _data.Info.MediaType,
                    'MsgBotInfo': _data.Info.MsgBotInfo,
                    'MsgMetaInfo': _data.Info.MsgMetaInfo,
                    'Multicast': _data.Info.Multicast,
                    'PushName': _data.Info.PushName,
                    'RecipientAlt': _data.Info.RecipientAlt,
                    'Sender': _data.Info.Sender,
                    'SenderAlt': _data.Info.SenderAlt,
                    'ServerID': _data.Info.ServerID,
                    'Timestamp': _data.Info.Timestamp,
                    'Type': _data.Info.Type,
                    'VerifiedName': _data.Info.VerifiedName
                }},
                'Message': _data.Message,
                'IsEphemeral': _data.IsEphemeral,
                'IsViewOnce': _data.IsViewOnce,
                'IsViewOnceV2': _data.IsViewOnceV2,
                'IsViewOnceV2Extension': _data.IsViewOnceV2Extension,
                'IsDocumentWithCaption': _data.IsDocumentWithCaption,
                'IsLottieSticker': _data.IsLottieSticker,
                'IsBotInvoke': _data.IsBotInvoke,
                'IsEdit': _data.IsEdit,
                'SourceWebMsg': _data.SourceWebMsg,
                'UnavailableRequestID': _data.UnavailableRequestID,
                'RetryCount': _data.RetryCount,
                'NewsletterMeta': _data.NewsletterMeta,
                'RawMessage': _data.RawMessage,
                'IsReal': _data.IsReal,
                'Status': _data.Status
            }} AS _data
        FROM read_json_auto('{path}', ignore_errors=true)
    ) TO 'bronze/bronze_messages_{data_atual}.parquet' (FORMAT PARQUET, COMPRESSION SNAPPY)
''')

# Lê de volta para exibir
df = con.execute(f"SELECT * FROM 'bronze/bronze_messages_{data_atual}.parquet'").df()
df

,id,timestamp,from,fromMe,source,body,to,participant,hasMedia,media,ack,ackName,replyTo,_data
0,false_120363045152243726@g.us_2A162697F84CB640...,1771848130,120363045152243726@g.us,False,app,CUPOM LIBERADO MELI 🟡\n\nCorreeee\n\n🏷️ *COMPR...,117180196749398@lid,117180196749398@lid,True,None,NaN,UNKNOWN,NaN,"{'Info': {'AddressingMode': '', 'BroadcastList..."
1,false_120363045152243726@g.us_2AF8AA60A7C0BD86...,1771770513,120363045152243726@g.us,False,app,CHUVA DE BRINDES 🚨🚨🚨\n\n👉🏻 Ganhe 4 miniaturas ...,117180196749398@lid,117180196749398@lid,True,None,NaN,UNKNOWN,NaN,"{'Info': {'AddressingMode': '', 'BroadcastList..."
2,false_120363045152243726@g.us_2A801E28EA6EEE89...,1771769350,120363045152243726@g.us,False,app,A MELHOR PROMO VOLTOU 🚨🚨🚨\n\nLeve 10 minis *da...,117180196749398@lid,117180196749398@lid,True,None,NaN,UNKNOWN,NaN,"{'Info': {'AddressingMode': '', 'BroadcastList..."
3,false_120363045152243726@g.us_2A988DBFA85AEEA0...,1771769348,120363045152243726@g.us,False,app,Ganhe *SHAMPOO FULL SIZE - 300ML* nas compra d...,117180196749398@lid,117180196749398@lid,True,None,NaN,UNKNOWN,NaN,"{'Info': {'AddressingMode': '', 'BroadcastList..."
4,false_120363045152243726@g.us_2A4A1F4D0D21588B...,1771769345,120363045152243726@g.us,False,app,Ganhe *15% OFF EXTRA* nas marcas _Giorgio Arma...,117180196749398@lid,117180196749398@lid,True,None,NaN,UNKNOWN,NaN,"{'Info': {'AddressingMode': '', 'BroadcastList..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2064,false_556692243055-1614900270@g.us_ACDFCA7630E...,1771719828,556692243055-1614900270@g.us,False,app,Tipo uma dessa ?,45483854684162@lid,45483854684162@lid,False,None,NaN,UNKNOWN,NaN,"{'Info': {'AddressingMode': '', 'BroadcastList..."
2065,false_556692243055-1614900270@g.us_ACBAA43B5E5...,1771719825,556692243055-1614900270@g.us,False,app,None,45483854684162@lid,45483854684162@lid,True,None,NaN,UNKNOWN,NaN,"{'Info': {'AddressingMode': '', 'BroadcastList..."
2066,false_556692243055-1614900270@g.us_AC1AEBF77DA...,1771719820,556692243055-1614900270@g.us,False,app,"Sim,",165674219487432@lid,165674219487432@lid,False,None,NaN,UNKNOWN,"{'id': 'AC56AD11DC6C571072E55398BEC9FAAD', 'pa...","{'Info': {'AddressingMode': '', 'BroadcastList..."
2067,false_556692243055-1614900270@g.us_AC56AD11DC6...,1771719808,556692243055-1614900270@g.us,False,app,Alguém aqui usa uma manopla de macha diferente?,45483854684162@lid,45483854684162@lid,False,None,NaN,UNKNOWN,NaN,"{'Info': {'AddressingMode': '', 'BroadcastList..."


In [47]:
#CAMADA SILVER
path = r'bronze\*.parquet'
df = con.execute('''
    select _data.Info from read_parquet(?)
''', [path]).df()

# df.to_parquet(f'silver/silver_messages_{data_atual}.parquet', index=False, compression='snappy')
df

,Info
0,"{'AddressingMode': '', 'BroadcastListOwner': '..."
1,"{'AddressingMode': '', 'BroadcastListOwner': '..."
2,"{'AddressingMode': '', 'BroadcastListOwner': '..."
3,"{'AddressingMode': '', 'BroadcastListOwner': '..."
4,"{'AddressingMode': '', 'BroadcastListOwner': '..."
...,...
2064,"{'AddressingMode': '', 'BroadcastListOwner': '..."
2065,"{'AddressingMode': '', 'BroadcastListOwner': '..."
2066,"{'AddressingMode': '', 'BroadcastListOwner': '..."
2067,"{'AddressingMode': '', 'BroadcastListOwner': '..."


In [4]:
#CAMADA SILVER
path = r'bronze\*.parquet'
df = con.execute('''
    SELECT max(timestamp) as max_timestamp, min(timestamp) as min_timestamp
                 
    FROM read_parquet(?)
''', [path]).df()

# df.to_parquet(f'silver/silver_messages_{data_atual}.parquet', index=False, compression='snappy')
df

,max_timestamp,min_timestamp
0,1770123469,1769990546


In [7]:
#CAMADA SILVER
path = r'bronze\*.parquet'
df = con.execute('''
    with dados as (SELECT id,
    strftime(to_timestamp(cast(timestamp as bigint)), '%Y-%m-%d') as date,
    strftime(to_timestamp(cast(timestamp as bigint)), '%H') as time
                 
    FROM read_parquet(?)
                 )
    select distinct date, time, count(id) as total_messages
    from dados
    group by date, time
    order by date, time
''', [path]).df()

df.to_csv(f'silver/val_silver_messages_{data_atual}.csv', index=False)
df

,date,time,total_messages
0,2026-02-01,21,47
1,2026-02-01,22,23
2,2026-02-01,23,26
3,2026-02-02,00,275
4,2026-02-02,01,110
5,2026-02-02,02,5
6,2026-02-02,06,2
7,2026-02-02,07,1
8,2026-02-02,08,6
9,2026-02-02,09,17


In [ ]:
# #CAMADA SILVER
# path = r'bronze\*.parquet'
# df = con.execute('''
#     SELECT DISTINCT
#         id,
#         strftime(to_timestamp(cast(timestamp as bigint)), '%Y-%m-%d') as date,
#         strftime(to_timestamp(cast(timestamp as bigint)), '%H:%M:%S') as time,
#         substring((split(cast(_data.Info.SenderAlt as varchar),':'))[1], 0, 3) as country_code,
#         substring((split(cast(_data.Info.SenderAlt as varchar),':'))[1], 3, 2) as state_code,
#         substring((split(cast(_data.Info.SenderAlt as varchar),':'))[1], 5, 9).replace('@', '') as tel_number,
#         body.replace('\n', ' ').replace('\r', ' ') as body,
#         cast(_data.Message.extendedTextMessage.text as varchar).replace('\n', ' ').replace('\r', ' ') as text,
#         cast(_data.Message.extendedTextMessage.title as varchar).replace('\n', ' ').replace('\r', ' ') as title,
#         cast(_data.Message.extendedTextMessage.description as varchar).replace('\n', ' ').replace('\r', ' ') as description,
#         cast(_data.Message.imageMessage.caption as varchar).replace('\n', ' ').replace('\r', ' ') as caption
                 
#     FROM read_parquet(?)
#     WHERE (body like '%http%' OR body like '%www.%')
#     AND (body not like '%whatsapp%')
# ''', [path]).df()

# # df.to_parquet(f'silver/silver_messages_{data_atual}.parquet', index=False, compression='snappy')
# df

BinderException: Binder Error: Could not find key "extendedtextmessage" in struct

Candidate Entries: "imageMessage"

In [ ]:
# #CAMADA GOLD
# path = r'silver\*.parquet'
# groups = r'groups.json'
# path_base_excel = 'cadastro_grupos\Share of Voice - Grupos_TRATADO.xlsx'
# df = con.execute('''
#     WITH parte1 as (
#     SELECT DISTINCT
#         silver.id, silver.date, silver.time, silver.country_code, silver.state_code, silver.tel_number, silver.body, silver.text, silver.title, silver.description, silver.caption,
#         case
#         -- ECOMMERCE
#         when lower(body) like '%mercadolivre%' or lower(text) like '%mercadolivre%' or lower(title) like '%mercadolivre%' or lower(description) like '%mercadolivre%' or lower(caption) like '%mercadolivre%' then 'MERCADO LIVRE'
#         when lower(body) like '%magalu%' or lower(text) like '%magalu%' or lower(title) like '%magalu%' or lower(description) like '%magalu%' or lower(caption) like '%magalu%' then 'MAGAZINE LUIZA'
#         when lower(body) like '%shopee%' or lower(text) like '%shopee%' or lower(title) like '%shopee%' or lower(description) like '%shopee%' or lower(caption) like '%shopee%' then 'SHOPEE'
#         when lower(body) like '%amzn%' or lower(text) like '%amzn%' or lower(title) like '%amzn%' or lower(description) like '%amzn%' or lower(caption) like '%amzn%' then 'AMAZON'
#         when lower(body) like '%amazon%' or lower(text) like '%amazon%' or lower(title) like '%amazon%' or lower(description) like '%amazon%' or lower(caption) like '%amazon%' then 'AMAZON'
#         when lower(body) like '%shein%' or lower(text) like '%shein%' or lower(title) like '%shein%' or lower(description) like '%shein%' or lower(caption) like '%shein%' then 'SHEIN'
#         -- AGREGADORES
#         when lower(body) like '%achadosprincipais%' or lower(text) like '%achadosprincipais%' or lower(title) like '%achadosprincipais%' or lower(description) like '%achadosprincipais%' or lower(caption) like '%achadosprincipais%' then 'AGREGADOR'
#         when lower(body) like '%promorelampago%' or lower(text) like '%promorelampago%' or lower(title) like '%promorelampago%' or lower(description) like '%promorelampago%' or lower(caption) like '%promorelampago%' then 'AGREGADOR'
#         when lower(body) like '%minhaloja.%' or lower(text) like '%minhaloja.%' or lower(title) like '%minhaloja.%' or lower(description) like '%minhaloja.%' or lower(caption) like '%minhaloja.%' then 'AGREGADOR'
#         -- BETS
#         when lower(body) like '%aa7.%' or lower(text) like '%aa7.%' or lower(title) like '%aa7.%' or lower(description) like '%aa7.%' or lower(caption) like '%aa7.%' then 'BETS'
#         when lower(body) like '%bmw7.%' or lower(text) like '%bmw7.%' or lower(title) like '%bmw7.%' or lower(description) like '%bmw7.%' or lower(caption) like '%bmw7.%' then 'BETS'
#         else 'OUTROS' end as origin,
#         case
#         when lower(body) like '%cupom%' or lower(text) like '%cupom%' or lower(title) like '%cupom%' or lower(description) like '%cupom%' or lower(caption) like '%cupom%' then True
#         else False end as coupon,
#         coalesce(base_origem.nome_grupo, 'N/A') as group_name,
#         coalesce(base_origem.class_atual, 'N/A') as class
#     FROM read_parquet(?) as silver
#     LEFT JOIN read_xlsx(?, all_varchar=True) AS base_origem
#     ON ((split(silver.id, '_'))[2] = base_origem.id_api
#     AND base_origem.tipo_link in ('Grupo'))
#     )
                 
#     SELECT id, date, time, country_code, state_code, tel_number, origin, coupon, group_name, class,
#     case
#     when origin <> 'BETS' then consultar_chatgpt(cast(concat(coalesce(body, 'N/A'), ' - ', coalesce(text, 'N/A'), ' - ', coalesce(title, 'N/A'), ' - ', coalesce(description, 'N/A'), ' - ', coalesce(caption, 'N/A')) as varchar))
#     -- when origin <> 'BETS' then id -- PARA TESTE
#     else 'N/A' end as category
#     FROM parte1
# ''', [path, path_base_excel]).df()

# df.head(5)
# df.to_parquet(f'gold/gold_messages_{data_atual}.parquet', index=False, compression='snappy')
# df.to_csv(f'gold/gold_messages_{data_atual}.csv', index=False, sep=';')

In [ ]:
# #CAMADA GOLD
# path = r'silver\*.parquet'
# groups = r'groups.json'
# df = con.execute('''
#     WITH parte1 as (
#     SELECT DISTINCT
#         silver.id, silver.date, silver.time, silver.country_code, silver.state_code, silver.tel_number, silver.body, silver.text, silver.title, silver.description, silver.caption,
#         case
#         -- ECOMMERCE
#         when lower(body) like '%mercadolivre%' or lower(text) like '%mercadolivre%' or lower(title) like '%mercadolivre%' or lower(description) like '%mercadolivre%' or lower(caption) like '%mercadolivre%' then 'MERCADO LIVRE'
#         when lower(body) like '%magalu%' or lower(text) like '%magalu%' or lower(title) like '%magalu%' or lower(description) like '%magalu%' or lower(caption) like '%magalu%' then 'MAGAZINE LUIZA'
#         when lower(body) like '%shopee%' or lower(text) like '%shopee%' or lower(title) like '%shopee%' or lower(description) like '%shopee%' or lower(caption) like '%shopee%' then 'SHOPEE'
#         when lower(body) like '%amzn%' or lower(text) like '%amzn%' or lower(title) like '%amzn%' or lower(description) like '%amzn%' or lower(caption) like '%amzn%' then 'AMAZON'
#         when lower(body) like '%amazon%' or lower(text) like '%amazon%' or lower(title) like '%amazon%' or lower(description) like '%amazon%' or lower(caption) like '%amazon%' then 'AMAZON'
#         when lower(body) like '%shein%' or lower(text) like '%shein%' or lower(title) like '%shein%' or lower(description) like '%shein%' or lower(caption) like '%shein%' then 'SHEIN'
#         -- AGREGADORES
#         when lower(body) like '%achadosprincipais%' or lower(text) like '%achadosprincipais%' or lower(title) like '%achadosprincipais%' or lower(description) like '%achadosprincipais%' or lower(caption) like '%achadosprincipais%' then 'AGREGADOR'
#         when lower(body) like '%promorelampago%' or lower(text) like '%promorelampago%' or lower(title) like '%promorelampago%' or lower(description) like '%promorelampago%' or lower(caption) like '%promorelampago%' then 'AGREGADOR'
#         when lower(body) like '%minhaloja.%' or lower(text) like '%minhaloja.%' or lower(title) like '%minhaloja.%' or lower(description) like '%minhaloja.%' or lower(caption) like '%minhaloja.%' then 'AGREGADOR'
#         -- BETS
#         when lower(body) like '%aa7.%' or lower(text) like '%aa7.%' or lower(title) like '%aa7.%' or lower(description) like '%aa7.%' or lower(caption) like '%aa7.%' then 'BETS'
#         when lower(body) like '%bmw7.%' or lower(text) like '%bmw7.%' or lower(title) like '%bmw7.%' or lower(description) like '%bmw7.%' or lower(caption) like '%bmw7.%' then 'BETS'
#         else 'OUTROS' end as origin,
#         case
#         when lower(body) like '%cupom%' or lower(text) like '%cupom%' or lower(title) like '%cupom%' or lower(description) like '%cupom%' or lower(caption) like '%cupom%' then True
#         else False end as coupon,
#         coalesce(groups.name, 'N/A') as group_name
#     FROM read_parquet(?) as silver
#     LEFT JOIN read_json_auto(?) AS groups
#     ON (split(silver.id, '_'))[2] = groups.id
#     )
                 
#     SELECT id, date, time, country_code, state_code, tel_number, origin, coupon, group_name,
#     case
#     when origin <> 'BETS' then consultar_chatgpt(cast(concat(coalesce(body, 'N/A'), ' - ', coalesce(text, 'N/A'), ' - ', coalesce(title, 'N/A'), ' - ', coalesce(description, 'N/A'), ' - ', coalesce(caption, 'N/A')) as varchar))
#     else 'N/A' end as category
#     FROM parte1
# ''', [path, groups]).df()

# df.head(2)
# # df.to_parquet(f'gold/gold_messages_{data_atual}.parquet', index=False, compression='snappy')
# # df.to_csv(f'gold/gold_messages_{data_atual}.csv', index=False)